# Import libraries

In [23]:
import pandas as pd
import numpy as np

# Load data

In [24]:
df = pd.read_csv('in-vehicle-coupon-recommendation.csv')
df.head()

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,No Urgent Place,Alone,Sunny,55,2PM,Restaurant(<20),1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,0,1,1
1,No Urgent Place,Friend(s),Sunny,80,10AM,Coffee House,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,0,1,0
2,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,1
3,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0
4,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0


# Build a Decision Tree model

In [25]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Preprocessing

In [26]:
df = df.drop(['car'], axis=1)

In [27]:
df = df.dropna()

In [29]:
df = pd.get_dummies(df)
df.head()

,temperature,has_children,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y,destination_Home,destination_No Urgent Place,...,RestaurantLessThan20_1~3,RestaurantLessThan20_4~8,RestaurantLessThan20_gt8,RestaurantLessThan20_less1,RestaurantLessThan20_never,Restaurant20To50_1~3,Restaurant20To50_4~8,Restaurant20To50_gt8,Restaurant20To50_less1,Restaurant20To50_never
22,55,0,1,0,0,0,1,1,0,1,...,0,1,0,0,0,0,0,0,1,0
23,80,0,1,0,0,0,1,0,0,1,...,0,1,0,0,0,0,0,0,1,0
24,80,0,1,0,0,0,1,1,0,1,...,0,1,0,0,0,0,0,0,1,0
25,80,0,1,1,0,0,1,0,0,1,...,0,1,0,0,0,0,0,0,1,0
26,80,0,1,0,0,0,1,0,0,1,...,0,1,0,0,0,0,0,0,1,0


In [30]:
X = df.drop(['Y'], axis=1)

In [31]:
y = df['Y'] 

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [37]:
# Function to test each column
def test_column(col):
    model = DecisionTreeClassifier(max_depth=4, random_state=123)
    model.fit(X_train[col].to_frame(), y_train)
    y_pred = model.predict(X_test[col].to_frame())
    return accuracy_score(y_test, y_pred)

In [41]:
def test_columns(cols):
    model = DecisionTreeClassifier(max_depth=4, random_state=123)
    model.fit(X_train[cols], y_train)
    y_pred = model.predict(X_test[cols])
    return accuracy_score(y_test, y_pred)

In [43]:
def test_all():
    model = DecisionTreeClassifier(max_depth=4, random_state=123)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

In [38]:
# Test each column
for column in X_train.columns:
    print(column, test_column(column))

temperature 0.581953642384106
has_children 0.581953642384106
toCoupon_GEQ5min 0.581953642384106
toCoupon_GEQ15min 0.581953642384106
toCoupon_GEQ25min 0.5885761589403974
direction_same 0.581953642384106
direction_opp 0.581953642384106
destination_Home 0.581953642384106
destination_No Urgent Place 0.581953642384106
destination_Work 0.5691225165562914
passanger_Alone 0.581953642384106
passanger_Friend(s) 0.581953642384106
passanger_Kid(s) 0.5753311258278145
passanger_Partner 0.581953642384106
weather_Rainy 0.5935430463576159
weather_Snowy 0.5811258278145696
weather_Sunny 0.5927152317880795
time_10AM 0.581953642384106
time_10PM 0.581953642384106
time_2PM 0.581953642384106
time_6PM 0.581953642384106
time_7AM 0.5691225165562914
coupon_Bar 0.6026490066225165
coupon_Carry out & Take away 0.581953642384106
coupon_Coffee House 0.581953642384106
coupon_Restaurant(20-50) 0.5968543046357616
coupon_Restaurant(<20) 0.581953642384106
expiration_1d 0.5496688741721855
expiration_2h 0.5496688741721855
ge

In [42]:
# Test columns in pairs
for col1 in X_train.columns:
    for col2 in X_train.columns:
        if col1 < col2:
            print(col1, col2, test_columns([col1, col2]))

temperature toCoupon_GEQ5min 0.581953642384106
temperature toCoupon_GEQ15min 0.5918874172185431
temperature toCoupon_GEQ25min 0.5894039735099338
temperature weather_Rainy 0.5935430463576159
temperature weather_Snowy 0.5811258278145696
temperature weather_Sunny 0.5927152317880795
temperature time_10AM 0.5658112582781457
temperature time_10PM 0.5761589403973509
temperature time_2PM 0.5923013245033113
temperature time_6PM 0.581953642384106
temperature time_7AM 0.5889900662251656
has_children temperature 0.5807119205298014
has_children toCoupon_GEQ5min 0.581953642384106
has_children toCoupon_GEQ15min 0.5790562913907285
has_children toCoupon_GEQ25min 0.5885761589403974
has_children passanger_Alone 0.581953642384106
has_children passanger_Friend(s) 0.581953642384106
has_children passanger_Kid(s) 0.5761589403973509
has_children passanger_Partner 0.581953642384106
has_children weather_Rainy 0.5935430463576159
has_children weather_Snowy 0.5811258278145696
has_children weather_Sunny 0.5927152317

In [44]:
# Accuracy of all columns
test_all()

0.6576986754966887